In [ ]:
# Importing Important Libraries

import pandas as pd

In [ ]:
# Installing datasets library for loading big-clone-bench dataset

!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.2 MB/s eta 0:00:00


In [ ]:
# Loading the Big-Clone-Bench dataset

from datasets import load_dataset
dataset = load_dataset("code_x_glue_cc_clone_detection_big_clone_bench")

Generating train split:   0%|          | 0/901028 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/415416 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/415416 [00:00<?, ? examples/s]

In [ ]:
# Printing the keys in the dataset

print(dataset.keys())

dict_keys(['train', 'validation', 'test'])


In [ ]:
# Copying the model training data from the dataset into a dataframe

df = pd.DataFrame(dataset['train'])

In [ ]:
df = df.head(25000)

In [ ]:
# Printing the dataframe

df.head()

,id,id1,id2,func1,func2,label
0,0,13988825,8660836,private void setNodekeyInJsonResponse(Stri...,"public void transform(String style, String...",False
1,1,80378,18548122,public static void test(String args[]) {\n...,private static String loadUrlToString(Stri...,True
2,2,21354223,7421563,public String kodetu(String testusoila) {\...,private StringBuffer encoder(String arg) {...,True
3,3,15826299,19728871,public static void printResponseHeaders(St...,public static String getEncodedPassword(St...,False
4,4,9938081,11517213,public void load(String fileName) {\n ...,private static void copyFile(File sourceFi...,False


In [ ]:
# Dropping the unnecessary columns

column = ['id', 'id1', 'id2']

df = df.drop(column, axis = 1)

In [ ]:
# Converting the boolean labels into 0 and 1

df['label'] = df['label'].astype(int)

In [ ]:
# Printing the dataframe after deleting the columns

df.head()

,func1,func2,label
0,private void setNodekeyInJsonResponse(Stri...,"public void transform(String style, String...",0
1,public static void test(String args[]) {\n...,private static String loadUrlToString(Stri...,1
2,public String kodetu(String testusoila) {\...,private StringBuffer encoder(String arg) {...,1
3,public static void printResponseHeaders(St...,public static String getEncodedPassword(St...,0
4,public void load(String fileName) {\n ...,private static void copyFile(File sourceFi...,0


In [ ]:
# Printing the size of the dataframe

df.shape

(25000, 3)

In [ ]:
# Checking if there are any null values [if null values exist, further processing is required]

df.isnull().sum().sort_values(ascending=False)

func1    0
func2    0
label    0
dtype: int64

In [ ]:
# Decision Tree

from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

code_snippets = (df['func1'] + ' ' + df['func2']).tolist()

labels = df['label'].values

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(code_snippets)

X_train, X_test, y_train, y_test = train_test_split(
    X, labels, test_size=0.2, random_state=42
)

dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(X_train, y_train)

y_pred = dt_model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.963
Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.97      0.96      2449
           1       0.97      0.96      0.96      2551

    accuracy                           0.96      5000
   macro avg       0.96      0.96      0.96      5000
weighted avg       0.96      0.96      0.96      5000

